In [15]:
# get source index based on SSTc2d/SSTgb id
SSTid = 'J032800.3+300801'
from astropy.io import ascii

# Table: Properties of the c2d+GB YSOs
# ================================================================================
# Byte-by-byte Description of file: apjs519129t2_mrt.txt
# --------------------------------------------------------------------------------
#    Bytes Format Units  Label   Explanations
# --------------------------------------------------------------------------------
#    1-  4 I4     ---    Index   Index
#    6- 21 A16    ---    Cloud   Cloud identifier
#   23- 38 A16    ---    ID      SSTc2d or SSTgb + Spitzer source name  
#   40- 43 F4.1   mag    AV      The V band extinction 
#   45- 51 F7.2   ---    alpha   Observed spectral index
#   53- 58 F6.1   K      Tbol    Observed bolometric temperature
#   60- 69 F10.5  solLum Lbol    Observed bolometric luminosity
#   71- 77 F7.2   ---    alpha'  Extinction corrected spectral index
#   79- 84 F6.1   K      Tbol'   Extinction corrected bolometric temperature
#   86- 95 F10.5  solLum Lbol'   Extinction corrected bolometric luminosity
#   97- 97 A1     ---    AGB?    A likely AGB? Y or N
#   99- 99 A1     ---    Core?   Source has a core?
# --------------------------------------------------------------------------------


filename = '/Users/yaolun/data/Dunham15_c2dGB/apjs519129t2_mrt.txt'
data = ascii.read(filename, data_start=24, delimiter=' ',
                  names=['Index','Cloud','SSTid','Av','alpha','T_bol','L_bol',
                         'alpha_extcor','T_bol_extcor','L_bol_extcor','AGB_flag','Core_flag'])

data[data['SSTid'] == SSTid]

Index,Cloud,SSTid,Av,alpha,T_bol,L_bol,alpha_extcor,T_bol_extcor,L_bol_extcor,AGB_flag,Core_flag
int64,str16,str16,float64,float64,float64,float64,float64,float64,float64,str1,str1
2369,Perseus,J032800.3+300801,5.9,0.95,230.0,0.25,0.98,280.0,0.29,N,Y


In [30]:
# get IRAC fluxes out from Dunham+16 table 2 for given source index
from astropy.io import ascii
from pprint import pprint

# ================================================================================
# Byte-by-byte Description of file: apjs519129t4_mrt.txt
# --------------------------------------------------------------------------------
#    Bytes Format Units Label  Explanations
# --------------------------------------------------------------------------------
#    1-  4 I4     ---   Index  Index 
#    6- 13 F8.2   mJy   F1.25  J band 1.25 micron flux density
#   15- 24 F10.4  mJy  eF1.25  Uncertainty in F1.25
#   26- 34 F9.3   mJy   F1.65  H band 1.65 micron flux density
#   36- 44 F9.4   mJy  eF1.65  Uncertainty in F1.65
#   46- 54 F9.2   mJy   F2.17  K_S_ band 2.17 micron flux density 
#   56- 64 F9.3   mJy  eF2.17  Uncertainty in F2.17
#   66- 74 F9.4   mJy   F3.6   Spitzer/IRAC 3.6 micron flux density
#   76- 84 F9.4   mJy  eF3.6   Uncertainty in F3.6
#   86- 93 F8.3   mJy   F4.5   Spitzer/IRAC 4.5 micron flux density
#   95-103 F9.4   mJy  eF4.5   Uncertainty in F4.5
#  105-112 F8.2   mJy   F5.8   Spitzer/IRAC 5.8 micron flux density
#  114-121 F8.3   mJy  eF5.8   Uncertainty in F5.8
#  123-130 F8.2   mJy   F8.0   Spitzer/IRAC 8.0 micron flux density
#  132-139 F8.3   mJy  eF8.0   Uncertainty in F8.0
#  141-148 F8.2   mJy   F24    Spitzer/MIPS 24 micron flux density
#  150-156 F7.2   mJy  eF24    Uncertainty in F24
#  158-163 I6     mJy   F70    Spitzer/MIPS 70 micron flux density
#  165-171 F7.1   mJy  eF70    Uncertainty in F70
# --------------------------------------------------------------------------------

# no extinction correction
# filename = '/Users/yaolun/data/Dunham15_c2dGB/apjs519129t3_mrt.txt'
# extinction corrected
filename = '/Users/yaolun/data/Dunham15_c2dGB/apjs519129t4_mrt.txt'
data = ascii.read(filename, data_start=31, names=['Index','F_J','err_J','F_H','err_H','F_Ks','err_Ks','F_IRAC1',
                                                  'err_IRAC1','F_IRAC2','err_IRAC2','F_IRAC3','err_IRAC3','F_IRAC4',
                                                  'err_IRAC4','F_MIPS24','err_MIPS24','F_MIPS70','err_MIPS70'])

data[data['Index'] == 1552]

Index,F_J,err_J,F_H,err_H,F_Ks,err_Ks,F_IRAC1,err_IRAC1,F_IRAC2,err_IRAC2,F_IRAC3,err_IRAC3,F_IRAC4,err_IRAC4,F_MIPS24,err_MIPS24,F_MIPS70,err_MIPS70
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64
1552,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,1.9,0.4,6.4,0.64,4.5,0.63,2.9,0.56,91.0,8.5,11000,1200.0


In [26]:
def IRACmag2flux(mag, mag_err):
    # calculate the absolute flux for IRAC given magnitude
    import numpy as np 
    # Jy
    zero_mag_flux = np.array([280.9, 179.7, 115.0, 64.9])
    zero_mag_err = np.array([4.1, 2.6, 1.7, 0.9])

    # formula: F_v = F_v,o * 10^(-2/5*m)
    # error propagation: f = a^(b*A); sigma_f^2 = f^2*(b*ln(a)*sigma_A)^2
    #                    f = AB; sigma_f^2 = (sigma_A/A)^2+(sigma_B/B)^2

    flux = zero_mag_flux * 10**(-2./5*mag)
    flux_err_expo = 10**(-2./5*mag)*((-2./5*np.log(10)*mag_err)**2)**0.5
    flux_err = ((zero_mag_err/zero_mag_flux)**2 + (flux_err_expo/10**(-2./5*mag))**2)**0.5

    return flux, flux_err

def MIPSmag2flux(mag, mag_err):
    # calculate the absolute flux for IRAC given magnitude
    import numpy as np 
    # Jy
    zero_mag_flux = np.array([7.17, 0.778, 0.159])
    zero_mag_err = np.array([0.11, 0.012, 0.020])

    # formula: F_v = F_v,o * 10^(-2/5*m)
    # error propagation: f = a^(b*A); sigma_f^2 = f^2*(b*ln(a)*sigma_A)^2
    #                    f = AB; sigma_f^2 = (sigma_A/A)^2+(sigma_B/B)^2

    flux = zero_mag_flux * 10**(-2./5*mag)
    flux_err_expo = 10**(-2./5*mag)*((-2./5*np.log(10)*mag_err)**2)**0.5
    flux_err = ((zero_mag_err/zero_mag_flux)**2 + (flux_err_expo/10**(-2./5*mag))**2)**0.5

    return flux, flux_err

mag = np.array([9.15, 7.76, 6.72, 5.75])
mag_err = np.array([0.05, 0.05, 0.05, 0.05])
flux, flux_err = IRACmag2flux(mag, mag_err)

print('IRAC')
print(flux)
print(flux_err)

mag = np.array([1.43, -1.92, -4.33])
mag_err = np.array([0.04, 0.22, 0.34])
flux, flux_err = MIPSmag2flux(mag, mag_err)

print('MIPS')
print(flux)
print(flux_err)

IRAC
[ 0.06145422  0.14143213  0.23588365  0.32527051]
[ 0.04830943  0.04827109  0.04836615  0.04809435]
MIPS
[ 1.92096369  4.56015492  8.5782189 ]
[ 0.03990807  0.20321369  0.3374701 ]


In [ ]:
9.15 ± 0.05 7.76 ± 0.05 6.72 ± 0.05 5.75 ± 0.05 1.43 ± 0.04 −1.92 ± 0.22 −4.33 ± 0.34